In [ ]:
import pandas as pd

df = pd.read_csv('/content/dataset_lemm.csv', encoding='utf-16')
df.tail()

,Unnamed: 0,category,text
1507,1507,other,нафтогаз заявить украинский сторона согласный ...
1508,1508,other,страна опек договориться снижение объём добыча...
1509,1509,other,меджлис крымскотатарский народ экстремистский ...
1510,1510,other,недавно вернуться россия певица наталья ветлиц...
1511,1511,other,канцлер германия ангела меркель впервые 14 год...


In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer
import torch
import numpy as np

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
labels = {
          'study':0,
          'other':1
          }

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1209 151 152


In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 2
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 605/605 [02:27<00:00,  4.10it/s]


Epochs: 1 | Train Loss:  0.276                 | Train Accuracy:  0.730                 | Val Loss:  0.187                 | Val Accuracy:  0.960


100%|██████████| 605/605 [02:27<00:00,  4.11it/s]


Epochs: 2 | Train Loss:  0.172                 | Train Accuracy:  0.962                 | Val Loss:  0.072                 | Val Accuracy:  0.987


In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

Test Accuracy:  0.993


In [ ]:
import torch

torch.save(model, '/content/model.pt')

In [ ]:
!pip install pymorphy2

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopword = stopwords.words('russian')

def remove_stopwords(text):
    words = text.split()
    ans = ''
    for word in words:
        if word not in stopword:
            ans += word + ' '
    
    return ans


import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form + ' ')

    return ''.join(res)



def predict(model, text):
    text = text.lower() #приводим к нижнему регистру
    text = remove_stopwords(text)
    text = lemmatize(text)
    print(text)

    tokenized = tokenizer(text, 
                padding='max_length', max_length = 512, truncation=True,
                return_tensors="pt")
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
    
    with torch.no_grad():
        mask = tokenized['attention_mask'].to(device)
        input_id = tokenized['input_ids'].squeeze(1).to(device)
        output = model(input_id, mask)
    
    print(output)
    if(output.argmax(dim=1) == 0): return 'study'
    if(output.argmax(dim=1) == 1): return 'other'




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
predict(model, 'гарри поттер моя любимая книга детства и вообще лучшее произведение десятилетия')

гарри поттер любимый книга детство вообще хороший произведение десятилетие 
tensor([[1.2244, 0.0000]], device='cuda:0')


'study'

In [ ]:
predict(model, 'Первая полностью российская шапочка из фольги\
                была выпущена на новосибирском заводе «Спецстроймонтаж». Директор предприятия \
                Алексей Джарахов назвал случившееся невероятным успехом, ведь ещё год назад 90% \
                комплектующих поставлялись из-за рубежа.')

первый полностью российский шапочка фольга выпустить новосибирский завод «спецстроймонтаж». директор предприятие алексей джараховый назвать случиться невероятный успехом, ещё год назад 90% комплектующие поставляться из-за рубежа. 
tensor([[0.1558, 0.2970]], device='cuda:0')


'other'

In [ ]:
predict(model, 'Первый канал предложил передать России победу на «Евровидении» из-за масштабных фальсификаций в ходе голосования')

первый канал предложить передать россия победа «евровидении» из-за масштабный фальсификация ход голосование 
tensor([[0.1286, 3.0647]], device='cuda:0')


'other'

In [ ]:
predict(model, 'В этом сезоне француз пробил отметку в 100 матчей за «Спартак». \
                Перед уходом в «Марсель» у игрока еще осталось одно незавершенное дело – финал Кубка России: \
                «У меня уже есть идея, как провести прощальный вечер с ребятами, но пока говорить об этом рано. \
                Сейчас у меня одна мечта – поднять трофей над головой, прежде чем отправиться во Францию».')

сезон француз пробить отметка 100 матч «спартак». уход «марсель» игрок остаться один незавершённый дело – финал кубок россии: «у идея, провести прощальный вечер ребятами, пока говорить рано. один мечта – поднять трофей головой, прежде отправиться францию». 
tensor([[0.2382, 0.2811]], device='cuda:0')


'other'

In [ ]:
predict(model, 'Наверное, никто не любит сдавать экзамены, но они являются неотъемлемой \
                частью жизни любого школьника или студента. Поэтому стоит потрудиться, чтобы \
                прийти на занятие подготовленным. В этом году свои знания будут защищать до 10,3 тысячи приморцев.\
                Основная часть из них – 9,4 тысячи – выпускники текущего года. Накануне экзаменов в министерстве образования \
                Приморского края напомнили основные правила сдачи ЕГЭ')

наверное, никто любить сдавать экзамены, являться неотъемлемый часть жизнь любой школьник студента. поэтому стоить потрудиться, прийти занятие подготовленным. год свой знание быть защищать 10,3 тысяча приморцев. основный часть – 9,4 тысяча – выпускник текущий года. накануне экзамен министерство образование приморский край напомнить основной правило сдача егэ 
tensor([[1.8199, 0.0000]], device='cuda:0')


'study'

In [ ]:
predict(model, 'по мнению опрошенных 31 февраля людей мирэа лучший университет в галактике')

мнение опросить 31 февраль человек мирэ хороший университет галактика 
tensor([[1.6238, 0.2853]], device='cuda:0')


'study'

In [ ]:
predict(model, 'по мнению опрошенных 31 февраля людей мирэа лучший институт в галактике')

мнение опросить 31 февраль человек мирэ хороший институт галактика 
tensor([[1.3715, 0.5009]], device='cuda:0')


'study'

In [ ]:
predict(model, 'по мнению опрошенных 31 февраля людей мирэа лучший в галактике')

мнение опросить 31 февраль человек мирэ хороший галактика 
tensor([[1.0137, 0.7147]], device='cuda:0')


'study'

In [ ]:
predict(model, 'по мнению опрошенных 31 февраля людей мирэа лучший институт в москве')

мнение опросить 31 февраль человек мирэ хороший институт москва 
tensor([[1.3093, 0.9033]], device='cuda:0')


'study'

In [ ]:
predict(model, 'мирэа лучший институт в москве')

мирэ хороший институт москва 
tensor([[1.4589, 0.0000]], device='cuda:0')


'study'

In [ ]:
predict(model, 'мирэа лучший в москве')

мирэ хороший москва 
tensor([[0.8389, 0.0000]], device='cuda:0')


'study'

In [ ]:
predict(model, 'бармен сказал что это самая лучшая книга по термодинамике которую он когда либо читал ')

бармен сказать это самый хороший книга термодинамика который либо читать 
tensor([[1.4665, 0.0000]], device='cuda:0')


'study'

In [ ]:
predict(model, 'в шашлычной у ашота бармен сказал мне что это самая лучшая книга по термодинамике которую он когда либо читал\
                после этого он взял шампур и сделал харакири')

шашлычный ашот бармен сказать это самый хороший книга термодинамика который либо читать взять шампур сделать харакири 
tensor([[1.8626, 0.0202]], device='cuda:0')


'study'

In [ ]:
predict(model, 'в шашлычной у ашота бармен сказал мне что это самая лучшая книга про эльфов которую он когда либо читал\
                после этого он взял шампур и сделал харакири')

шашлычный ашот бармен сказать это самый хороший книга эльф который либо читать взять шампур сделать харакири 
tensor([[1.6143, 0.1765]], device='cuda:0')


'study'

In [ ]:
predict(model, 'в шашлычной у ашота бармен сказал мне что это самая лучшая газета про эльфов которую он когда либо читал\
                после этого он взял шампур и сделал харакири')

шашлычный ашот бармен сказать это самый хороший газета эльф который либо читать взять шампур сделать харакири 
tensor([[1.4169, 0.3872]], device='cuda:0')


'study'

In [ ]:
predict(model, 'в шашлычной у ашота бармен сказал мне что это самая лучшая газета про эльфов которую он когда либо видел\
                после этого он взял шампур и сделал харакири')

шашлычный ашот бармен сказать это самый хороший газета эльф который либо видеть взять шампур сделать харакири 
tensor([[1.2055, 1.2228]], device='cuda:0')


'other'

In [ ]:
predict(model, 'я учусь в мирэа')

учиться мирэ 
tensor([[0.2556, 0.0000]], device='cuda:0')


'study'

In [ ]:
predict(model, 'мой одногруппник готовился к экзамену 90 секунд')

одногруппник готовиться экзамен 90 секунда 
tensor([[1.2404, 0.0000]], device='cuda:0')


'study'

In [ ]:
predict(model, 'Подскажите пожалуйста, кск принимает экзамен Соколова Л.А.')

подсказать пожалуйста, кск принимать экзамен соколов л.а. 
tensor([[0.4913, 0.0000]], device='cuda:0')


'study'